# 2D Injector Optimization with NN Prior at LCLS
Aiming to optimize transverse beam size in 2D

In [ ]:
import os
os.environ["OMP_NUM_THREADS"] = str(6)

In [ ]:
# optionally add scripts location to path
if True:
    import sys
    sys.path.append("../../")
    sys.path.append("../")

run_dir = "/home/physics3/ml_tuning/20231002_LCLS_Injector/"
print(sys.path)

## Set up image diagnostic

In [ ]:
from scripts.image import ImageDiagnostic
import yaml

fname = "../OTR3_config.yml"
image_diagnostic = ImageDiagnostic.parse_obj(yaml.safe_load(open(fname)))
image_diagnostic.save_image_location = run_dir
image_diagnostic.n_fitting_restarts = 2
image_diagnostic.visualize = False
image_diagnostic.background_file = run_dir + "OTRS_IN20_621_background.npy"
print(image_diagnostic.yaml())

In [ ]:
#image_diagnostic.measure_background(file_location=run_dir)

In [ ]:
image_diagnostic.background_file

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(image_diagnostic.background_image)

In [ ]:
image_diagnostic.test_measurement()

## Define VOCS

In [ ]:
import pandas as pd
from xopt import VOCS

In [ ]:
filename = "../variables.csv"
VARIABLE_RANGES = pd.read_csv(filename, index_col=0, header=None).T.to_dict(orient='list')

IMAGE_CONSTRAINTS = {
    "bb_penalty": ["LESS_THAN", 0.0],
    "log10_total_intensity": ["GREATER_THAN", image_diagnostic.min_log_intensity]
}

In [ ]:
VARIABLES = ["SOLN:IN20:121:BCTRL", "QUAD:IN20:121:BCTRL", "QUAD:IN20:122:BCTRL",
             "QUAD:IN20:361:BCTRL", "QUAD:IN20:371:BCTRL", "QUAD:IN20:425:BCTRL",
             "QUAD:IN20:441:BCTRL"]  # , "QUAD:IN20:511:BCTRL", "QUAD:IN20:525:BCTRL"

vocs = VOCS(
    variables = {ele: VARIABLE_RANGES[ele] for ele in VARIABLES},
    constraints = IMAGE_CONSTRAINTS,
    objectives = {"total_size": "MINIMIZE"},
)
print(vocs.as_yaml())

## Define evaluate function

In [ ]:
from time import sleep

import torch
import numpy as np
from epics import caput, caget_many

In [ ]:
def get_model_predictions(input_dict, generator = None):
    output_dict = {}
    for output_name in generator.vocs.output_names:
        if generator is not None and generator.data is not None and not generator.data.empty:
            gp = generator.model.models[generator.vocs.output_names.index(output_name)]
            x = torch.tensor([input_dict[k] for k in generator.vocs.variable_names], dtype=torch.double).unsqueeze(0)
            with torch.no_grad():
                _x = gp.input_transform.transform(x)
                _x = gp.mean_module(_x)
                prior_mean = gp.outcome_transform.untransform(_x)[0].item()
                posterior = gp.posterior(x)
                posterior_mean = posterior.mean.item()
                posterior_sd = torch.sqrt(posterior.mvn.variance).item()
        else:
            prior_mean, posterior_mean, posterior_sd = np.nan, np.nan, np.nan
    
        output_dict[output_name + "_prior_mean"] = prior_mean
        output_dict[output_name + "_posterior_mean"] = posterior_mean
        output_dict[output_name + "_posterior_sd"] = posterior_sd
    return output_dict

In [ ]:
def eval_beamsize(input_dict, generator = None):
    global image_diagnostic
    # set PVs
    for k, v in input_dict.items():
        print(f'CAPUT {k} {v}')
        caput(k, v)

    sleep(2.0)

    # get beam sizes from image diagnostic
    metadata = input_dict
    results = image_diagnostic.measure_beamsize(3, **metadata)
    results["S_x_mm"] = np.array(results["Sx"]) * 1e-3
    results["S_y_mm"] = np.array(results["Sy"]) * 1e-3

    # get other PV's NOTE: Measurements not synchronous with beamsize measurements!
    results = results

    # add total beam size
    results["total_size"] = np.sqrt(np.array(results["Sx"]) ** 2 + np.array(results["Sy"]) ** 2)
    # results["total_size"] = np.sqrt(np.abs(np.array(results["Sx"])) * np.array(results["Sy"]))
    
    # GP model predictions
    model_predictions = get_model_predictions(input_dict, generator)
    results.update(model_predictions)
    
    return results

## Define NN prior

In [ ]:
from lume_model.utils import variables_from_yaml
from lume_model.torch import LUMEModule, PyTorchModel

In [ ]:
model_path = "lcls_cu_injector_nn_model/"

# # load nn_to_cal transformers
# reg = "low"  # "low" or "high"
# input_nn_to_cal = torch.load(f"calibration/input_nn_to_cal_{reg}_reg.pt")
# output_nn_to_cal = torch.load(f"calibration/output_nn_to_cal_{reg}_reg.pt")

# load sim_to_nn transformers
input_sim_to_nn = torch.load(model_path + "model/input_sim_to_nn.pt")
output_sim_to_nn = torch.load(model_path + "model/output_sim_to_nn.pt")

# load pv_to_sim transformers
input_pv_to_sim = torch.load(model_path + "model/input_pv_to_sim.pt")
output_pv_to_sim = torch.load(model_path + "model/output_pv_to_sim.pt")

# load in- and output variable specification
input_variables, output_variables = variables_from_yaml(open(model_path + "model/pv_variables.yml"))
# input_variables, output_variables = variables_from_yaml(open(f"calibration/pv_variables_{reg}_reg.yml"))

# replace keys in input variables
input_variables = {name.replace("BACT", "BCTRL"): ele for name, ele in input_variables.items()}

# create LUME-model
lume_model = PyTorchModel(
    model_file=model_path + "model/model.pt",
    input_variables=input_variables,
    output_variables=output_variables,
    input_transformers=[input_pv_to_sim, input_sim_to_nn],
    output_transformers=[output_sim_to_nn, output_pv_to_sim],
    # input_transformers=[input_pv_to_sim, input_sim_to_nn, input_nn_to_cal],
    # output_transformers=[output_nn_to_cal, output_sim_to_nn, output_pv_to_sim],
)

# wrap in LUMEModule
lume_module = LUMEModule(
    model=lume_model,
    feature_order=vocs.variable_names,
    output_order=lume_model.outputs[0:2],
)

# define objective model
class ObjectiveModel(torch.nn.Module):
    def __init__(self, model: LUMEModule):
        super(ObjectiveModel, self).__init__()
        self.model = model

    @staticmethod
    def function(sigma_x: torch.Tensor, sigma_y: torch.Tensor) -> torch.Tensor:
        # using this calculation due to occasional negative values
        return torch.sqrt(sigma_x ** 2 + sigma_y ** 2)

    def forward(self, x) -> torch.Tensor:
        idx_sigma_x = self.model.output_order.index("OTRS:IN20:571:XRMS")
        idx_sigma_y = self.model.output_order.index("OTRS:IN20:571:YRMS")
        sigma_x = self.model(x)[..., idx_sigma_x]
        sigma_y = self.model(x)[..., idx_sigma_y]
        return self.function(sigma_x, sigma_y)

objective_model = ObjectiveModel(lume_module)

## Restrict ranges based on profile monitor

In [ ]:
vocs.variables = {k: input_variables[k].value_range for k in vocs.variable_names}
vocs.variables["SOLN:IN20:121:BCTRL"] = [0.467, 0.479]
print(vocs.as_yaml())

## Run Xopt

In [ ]:
from xopt import Xopt, VOCS
from xopt.evaluator import Evaluator
from xopt.numerical_optimizer import LBFGSOptimizer
from xopt.generators import ExpectedImprovementGenerator
from xopt.generators.bayesian.models.standard import StandardModelConstructor

# remember to set use low noise prior to false!!!
model_constructor = StandardModelConstructor(
    use_low_noise_prior=False,
    mean_modules={"total_size": objective_model},
)
generator = ExpectedImprovementGenerator(
    vocs=vocs,
    model_constructor=model_constructor,
    # turbo_controller="optimize"
)
generator.numerical_optimizer.max_iter = 200
evaluator = Evaluator(function=eval_beamsize, function_kwargs={"generator": generator})
X = Xopt(generator=generator, evaluator=evaluator, vocs=vocs)
X.dump_file = run_dir + "nn_optimization_7d_1.yml"
X

In [ ]:
X.random_evaluate(3, seed=0)

In [ ]:
%%time
for i in range(10):
    print(i)
    X.step()

In [ ]:
# X.data.to_csv(run_dir + "nn_optimization_7d_1.csv")

In [ ]:
X.data.plot(y="total_size")

In [ ]:
X.data.plot(y=X.vocs.variable_names)

In [ ]:
X.data[["total_size" + k for k in ["", "_prior_mean", "_posterior_mean", "_posterior_sd"]]].plot();

In [ ]:
from utils import plot_model_in_2d

fig, ax = plot_model_in_2d(
    X=X,
    output_name="total_size",  # should also work for constraints
    variable_names=None,  # provides option to view 2D slices at higher dimensions
    constrained_acqf=True,  # display the constrained or basic acquisition function
    n_grid=50,  # number of grid points per dimension
    figsize=(10,8),
    show_samples=True,
    fading_samples=True,  # older samples are more transparent
)

In [ ]:
fig, ax = plot_model_in_2d(
    X=X,
    output_name="bb_penalty",  # should also work for constraints
    variable_names=None,  # provides option to view 2D slices at higher dimensions
    constrained_acqf=False,  # display the constrained or basic acquisition function
    n_grid=50,  # number of grid points per dimension
    figsize=(10,8),
    show_samples=True,
    fading_samples=True,  # older samples are more transparent
)

In [ ]:
from scripts.utils.read_files import read_file
res = read_file(X.data.iloc[-1]["save_filename"])

In [ ]:
plt.imshow(res["images"][0])